In [ ]:
! cp ../local_settings.py .

In [ ]:
from pymongo import MongoClient

import os
# Variables about mongodb
MONGODB_HOST = os.environ.get("MONGODB_HOST", "localhost")
MONGODB_PORT = int(os.environ.get("MONGODB_PORT", 27017))
MONGODB_USER = os.environ.get("MONGODB_USER", "")
MONGODB_PWD = os.environ.get("MONGODB_PWD", "")
DB = os.environ.get("DB", "pyladies-linebots")

try:
    from local_settings import *
except:
    pass


mcli = MongoClient(host=MONGODB_HOST, port=MONGODB_PORT)
db = mcli[DB]

if MONGODB_USER!="":
    db.authenticate(MONGODB_USER,MONGODB_PWD)


In [ ]:
db.events.find_one()

In [ ]:
db.events.find_one()["f_line_id"]

In [ ]:
list(db.events.find())[0]["f_busy_time"]

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(list(db.events.find())[0]["matched_free_time"])

In [ ]:
list(db.events.find())[0]["my_busy_time"]

In [ ]:
list(db.events.find())[0]["f_busy_time"]

In [ ]:
time_range = "10D"
time_delta = "2H"
event_data = list(db.events.find())[0]

In [ ]:

import pytz
from datetime import datetime, timedelta
import math
import pandas as pd

def match_free_time(time_range, time_delta, event_data):
    assert time_range.endswith("D")
    assert isinstance(int(time_range.replace("D", "")), int)
    assert time_delta.endswith("H")
    assert isinstance(float(time_delta.replace("H", "")), float)

    time_range_days = int(time_range.replace("D", ""))
    time_delta_hours = float(time_delta.replace("H", ""))

    min_time_delta = 0.5
    time_delta_hours = math.ceil(time_delta_hours / min_time_delta) * min_time_delta
    n_sessions = math.ceil(time_delta_hours / min_time_delta)

    utc_now = datetime.utcnow()
    start_time = datetime(utc_now.year, utc_now.month, utc_now.day, utc_now.hour + 1)
    end_time = start_time + timedelta(days=time_range_days)

    twtz = pytz.timezone("Asia/Taipei")
    start_time = pytz.utc.localize(start_time).astimezone(twtz)
    end_time = pytz.utc.localize(end_time).astimezone(twtz)

    time_intervals_df = separate_into_xxxhours_sessions_for_listing(start_time, end_time, min_time_delta)
    time_intervals_df["all"] = 1
    
    return time_intervals_df

    busy_time = event_data["my_busy_time"]
    busy_time_df = pd.DataFrame(busy_time)
    
    if busy_time_df.shape[0] > 0:
    
        busy_time_df["end"] = busy_time_df.end.map(parse_utc_time_str_to_twtz_time)
        busy_time_df["start"] = busy_time_df.start.map(parse_utc_time_str_to_twtz_time)
        busy_time_df["time_delta_hours"] = min_time_delta
        busy_time_df["start"] = busy_time_df.start.map(
            lambda t: datetime(t.year, t.month, t.day, t.hour, 0 if t.minute < 30 else 30, 0, 0, t.tz))
        busy_time_df["end"] = busy_time_df.end.map(
            lambda t: datetime(t.year, t.month, t.day, t.hour + 1 if t.minute > 30 else t.hour,
                               0 if (t.minute > 30 or t.minute == 0) else 30, 0, 0, t.tz))
        
        my_time_df = pd.concat(
            list(map(lambda data: separate_into_xxxhours_sessions_for_listing(**data), busy_time_df.to_dict("records"))), 0)
        my_time_df["my"] = 0
        # my_time_df = my_time_df.set_index(["start","end"])
    
    else:
        my_time_df = time_intervals_df.copy()
        my_time_df["my"] = 1 
        my_time_df = f_time_df[["start","end","my"]]
        
        
    # return my_time_df
    
    busy_time = event_data["f_busy_time"]
    busy_time_df = pd.DataFrame(busy_time)
    print("busy_time_df = {v}".format(v=busy_time_df))
    print("busy_time_df.shape = {v}".format(v=busy_time_df.shape))
    
    if busy_time_df.shape[0] > 0:
        busy_time_df["end"] = busy_time_df.end.map(parse_utc_time_str_to_twtz_time)
        busy_time_df["start"] = busy_time_df.start.map(parse_utc_time_str_to_twtz_time)
        busy_time_df["time_delta_hours"] = min_time_delta

        busy_time_df["start"] = busy_time_df.start.map(
            lambda t: datetime(t.year, t.month, t.day, t.hour, 0 if t.minute < 30 else 30, 0, 0, t.tz))
        busy_time_df["end"] = busy_time_df.end.map(
            lambda t: datetime(t.year, t.month, t.day, t.hour + 1 if t.minute > 30 else t.hour,
                               0 if (t.minute > 30 or t.minute == 0) else 30, 0, 0, t.tz))

        f_time_df = pd.concat(
            list(map(lambda data: separate_into_xxxhours_sessions_for_listing(**data), busy_time_df.to_dict("records"))), 0)
        f_time_df["f"] = 0
        # f_time_df = f_time_df.set_index(["start","end"])
    else:
        f_time_df = time_intervals_df.copy()
        f_time_df["f"] = 1 
        f_time_df = f_time_df[["start","end","f"]]
        
    my_and_f_time_df = pd.merge(pd.merge(f_time_df, my_time_df, how="outer").fillna(1), time_intervals_df,
                                how="outer").fillna(1).sort(["start"], ascending=[1])
    
    #return my_and_f_time_df
    
    for i in range(n_sessions):
        my_and_f_time_df["my_{i}".format(i=i)] = my_and_f_time_df.my.shift(-i)
        my_and_f_time_df["f_{i}".format(i=i)] = my_and_f_time_df.f.shift(-i)

    my_cols = ["my_{i}".format(i=i) for i in range(n_sessions)]
    f_cols = ["f_{i}".format(i=i) for i in range(n_sessions)]

    my_and_f_time_df["my_cols_sum"] = my_and_f_time_df[my_cols].sum(1)
    my_and_f_time_df["f_cols_sum"] = my_and_f_time_df[f_cols].sum(1)

    df = my_and_f_time_df.set_index(["start", "end"])

    filtered_df = df[df[["my_cols_sum", "f_cols_sum"]].sum(1) == 2 * n_sessions].reset_index()[["start", "end"]]
    filtered_df["end"] = filtered_df.start.map(lambda t: t + timedelta(minutes=n_sessions * min_time_delta * 60))

    return filtered_df.to_dict("records")


def separate_into_xxxhours_sessions_for_listing(start, end, time_delta_hours, session_delta=30):
    time_intervals = []
    i = 0
    tmp_start_time = start
    tmp_end_time = start + timedelta(hours=time_delta_hours)

    while tmp_end_time <= end:
        time_intervals.append({"start": tmp_start_time, "end": tmp_end_time})
        tmp_start_time += timedelta(minutes=session_delta)
        tmp_end_time += timedelta(minutes=session_delta)
    time_intervals_df = pd.DataFrame(time_intervals)
    # time_intervals_df.columns = ["start","end"]

    return time_intervals_df


def separate_into_xxxhours_sessions_for_user(start, end, time_delta_hours, session_delta=30):
    time_intervals = []
    i = 0
    tmp_start_time = start
    tmp_end_time = start + timedelta(hours=time_delta_hours)

    while tmp_start_time < end:
        time_intervals.append({"start": tmp_start_time, "end": tmp_end_time})
        tmp_start_time += timedelta(minutes=session_delta)
        tmp_end_time += timedelta(minutes=session_delta)
    time_intervals_df = pd.DataFrame(time_intervals)
    # time_intervals_df.columns = ["start","end"]

    return time_intervals_df


def parse_utc_time_str_to_twtz_time(time_str):
    twtz = pytz.timezone("Asia/Taipei")
    t = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%SZ")
    t = pytz.utc.localize(t)
    t = t.astimezone(twtz)
    return t

In [ ]:
test = match_free_time(time_range, time_delta, event_data)
test